In [1]:
import cptac
import numpy as np
import pandas as pd
import statsmodels.stats.multitest as ssm

In [2]:
df = pd.read_csv('data/regressions_data.csv')
df

,gene,cancer,interaction_coeff,condition_coeff,transcript_coeff,intercept,interaction_pval,condition_pval,transcript_pval,intercept_pval
0,A1BG,ccrcc,-0.399310,0.683426,0.377849,-0.462140,0.028463,0.000251,1.453042e-02,4.968687e-03
1,A1CF,ccrcc,0.009264,-0.282587,0.073218,-0.563416,0.469640,0.007255,1.252619e-20,3.169151e-10
2,A2M,ccrcc,-0.001738,1.059763,0.001776,-0.899006,0.096824,0.000233,2.303762e-02,6.441968e-05
3,AAAS,ccrcc,-0.005048,0.253459,0.008609,-0.148476,0.702162,0.089684,2.967710e-01,2.751265e-01
4,AACS,ccrcc,-0.042658,0.313924,0.105237,-0.782216,0.064000,0.091160,1.406461e-09,6.938644e-07
...,...,...,...,...,...,...,...,...,...,...
51096,ZXDC,luad,-0.146323,0.889409,0.886971,-3.806726,0.927965,0.870308,4.850365e-01,4.176617e-01
51097,ZYG11B,luad,1.424677,-5.791951,-0.529182,2.529373,0.000585,0.000112,2.166567e-01,8.933655e-02
51098,ZYX,luad,0.714921,-5.450918,-0.123800,2.023549,0.017017,0.000068,7.125785e-01,1.238943e-01
51099,ZZEF1,luad,0.596868,-2.881060,0.511473,-1.305858,0.076980,0.022887,1.157354e-01,3.361588e-01


In [3]:
background_genes = list(pd.unique(df.gene))
len(background_genes)

12690

In [4]:
sig_df = df[df.interaction_pval <= 0.05]
sig_df

,gene,cancer,interaction_coeff,condition_coeff,transcript_coeff,intercept,interaction_pval,condition_pval,transcript_pval,intercept_pval
0,A1BG,ccrcc,-0.399310,0.683426,0.377849,-0.462140,0.028463,0.000251,1.453042e-02,4.968687e-03
8,AAK1,ccrcc,0.053062,-0.455378,0.032984,-0.009980,0.005954,0.000119,1.717676e-02,9.452946e-01
9,AAMP,ccrcc,0.012601,-0.242299,0.006251,-0.403301,0.015752,0.319595,1.604933e-01,4.028254e-02
12,AARS2,ccrcc,-0.083850,0.458319,0.152507,-1.035045,0.024073,0.089708,1.902464e-07,2.081794e-06
26,ABCB10,ccrcc,-0.203267,1.497990,0.283898,-2.063974,0.000262,0.000067,2.827046e-09,9.219280e-10
...,...,...,...,...,...,...,...,...,...,...
51094,ZWILCH,luad,3.158582,-4.319546,-0.801456,-0.265192,0.000018,0.000002,3.293234e-01,8.080739e-01
51095,ZWINT,luad,0.875887,-2.358641,0.188274,-0.835616,0.007286,0.000876,5.507354e-01,1.245294e-03
51097,ZYG11B,luad,1.424677,-5.791951,-0.529182,2.529373,0.000585,0.000112,2.166567e-01,8.933655e-02
51098,ZYX,luad,0.714921,-5.450918,-0.123800,2.023549,0.017017,0.000068,7.125785e-01,1.238943e-01


In [5]:
sig_genes = pd.DataFrame(sig_df.groupby('gene').size())
sig_genes = sig_genes[sig_genes[0] > 2]
sig_genes = list(sig_genes.index)
rows = []
for gene in sig_genes:
    row = dict()
    row['Gene'] = gene
    gene_df = sig_df[sig_df.gene == gene]
    avg_pval = np.mean(gene_df.interaction_pval)
    avg_interaction = np.mean(gene_df.interaction_coeff)
    abs_interaction = abs(avg_interaction)
    row['avg_pval'] = avg_pval
    row['avg_interaction_coeff'] = avg_interaction
    row['abs_interaction_coeff'] = abs_interaction
    rows.append(row)
sig_genes = pd.DataFrame(rows)
sig_genes

,Gene,avg_pval,avg_interaction_coeff,abs_interaction_coeff
0,AAK1,0.005085,0.453549,0.453549
1,AARS2,0.015352,0.719068,0.719068
2,AASDHPPT,0.001351,0.850032,0.850032
3,AASS,0.015262,1.079797,1.079797
4,ABCB10,0.013915,1.149501,1.149501
...,...,...,...,...
1364,ZNF638,0.002426,1.012607,1.012607
1365,ZNF687,0.014088,0.797145,0.797145
1366,ZNFX1,0.005495,0.388419,0.388419
1367,ZNHIT2,0.000306,0.756881,0.756881


In [6]:
sig_genes = sig_genes.sort_values('avg_pval')
pval_gene_list = list(sig_genes.Gene)
sig_genes = sig_genes.sort_values('abs_interaction_coeff', ascending = False)
interaction_gene_list = list(sig_genes.Gene)
positive_interactions = sig_genes[sig_genes.avg_interaction_coeff >= 0].sort_values('avg_interaction_coeff', ascending = False)
positive_interactions_gene_list = list(positive_interactions.Gene)
negative_interactions = sig_genes[sig_genes.avg_interaction_coeff <= 0].sort_values('avg_interaction_coeff')
negative_interactions_gene_list = list(negative_interactions.Gene)

In [7]:
from gprofiler import GProfiler
gp = GProfiler(return_dataframe=True)

In [8]:
pval_results_df = gp.profile(organism='hsapiens', query = pval_gene_list, no_iea=True,
                       ordered=True, no_evidences=False, background= background_genes, sources = ['KEGG'])

In [9]:
interaction_results_df = gp.profile(organism='hsapiens', query = interaction_gene_list, no_iea=True,
                       ordered=True, no_evidences=False, background= background_genes, sources = ['KEGG'])

In [10]:
down_reg_results_df = gp.profile(organism='hsapiens', query = negative_interactions_gene_list, no_iea=True,
                       ordered=True, no_evidences=False, background= background_genes, sources = ['KEGG'])

In [11]:
up_reg_results_df = gp.profile(organism='hsapiens', query = positive_interactions_gene_list, no_iea=True,
                       ordered=True, no_evidences=False, background= background_genes, sources = ['KEGG'])

In [12]:
pval_results_df = pval_results_df[pval_results_df.term_size >= 5]
pval_results_df = pval_results_df[pval_results_df.term_size <= 500]
pval_results_df = pval_results_df[pval_results_df.source == 'KEGG']
pval_results_df

,source,native,name,p_value,significant,description,term_size,query_size,intersection_size,effective_domain_size,precision,recall,query,parents,intersections,evidences
0,KEGG,KEGG:00520,Amino sugar and nucleotide sugar metabolism,0.000483,True,Amino sugar and nucleotide sugar metabolism,44,1331,17,12616,0.012772,0.386364,query_1,[KEGG:00000],"[UGDH, GFPT2, GFPT1, NPL, GMDS, NANP, UAP1, FP...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
1,KEGG,KEGG:01230,Biosynthesis of amino acids,0.000629,True,Biosynthesis of amino acids,73,1217,21,12616,0.017256,0.287671,query_1,[KEGG:00000],"[ALDOA, PYCR1, GOT1, BCAT1, PSPH, GPT2, SDS, C...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
2,KEGG,KEGG:00051,Fructose and mannose metabolism,0.000860,True,Fructose and mannose metabolism,32,1247,13,12616,0.010425,0.406250,query_1,[KEGG:00000],"[ALDOA, AKR1B10, FBP1, PFKFB2, GMDS, FPGT, PFK...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
3,KEGG,KEGG:00270,Cysteine and methionine metabolism,0.001641,True,Cysteine and methionine metabolism,47,1120,15,12616,0.013393,0.319149,query_1,[KEGG:00000],"[GOT1, BCAT1, AHCY, SDS, CTH, DNMT3A, MPST, IL...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
4,KEGG,KEGG:04115,p53 signaling pathway,0.001995,True,p53 signaling pathway,60,928,17,12616,0.018319,0.283333,query_1,[KEGG:00000],"[RRM2, SERPINE1, IGFBP3, CDK1, CASP9, STEAP3, ...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
5,KEGG,KEGG:00250,"Alanine, aspartate and glutamate metabolism",0.002082,True,"Alanine, aspartate and glutamate metabolism",33,1183,13,12616,0.010989,0.393939,query_1,[KEGG:00000],"[PPAT, GFPT2, GFPT1, GOT1, ADSL, GPT2, IL4I1, ...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
6,KEGG,KEGG:01200,Carbon metabolism,0.002199,True,Carbon metabolism,112,1247,27,12616,0.021652,0.241071,query_1,[KEGG:00000],"[ALDOA, GOT1, PSPH, GCSH, FBP1, GPT2, DLD, SDS...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
7,KEGG,KEGG:04512,ECM-receptor interaction,0.038173,True,ECM-receptor interaction,81,697,14,12616,0.020086,0.172840,query_1,[KEGG:00000],"[ITGAV, THBS2, CD36, ITGA11, ITGA2, COL1A1, CO...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
8,KEGG,KEGG:00030,Pentose phosphate pathway,0.040980,True,Pentose phosphate pathway,29,1217,10,12616,0.008217,0.344828,query_1,[KEGG:00000],"[ALDOA, FBP1, RPE, RBKS, PFKP, G6PD, PGM1, PFK...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."


In [13]:
interaction_results_df = interaction_results_df[interaction_results_df.term_size >= 5]
interaction_results_df = interaction_results_df[interaction_results_df.term_size <= 500]
interaction_results_df = interaction_results_df[interaction_results_df.source == 'KEGG']
interaction_results_df

,source,native,name,p_value,significant,description,term_size,query_size,intersection_size,effective_domain_size,precision,recall,query,parents,intersections,evidences
0,KEGG,KEGG:04115,p53 signaling pathway,0.000416,True,p53 signaling pathway,60,1017,19,12616,0.018682,0.316667,query_1,[KEGG:00000],"[CDKN2A, CDK1, CCNB1, CHEK1, SERPINE1, GTSE1, ...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
1,KEGG,KEGG:01230,Biosynthesis of amino acids,0.000492,True,Biosynthesis of amino acids,73,1097,20,12616,0.018232,0.273973,query_1,[KEGG:00000],"[RPIA, GPT2, PGK1, CTH, PSPH, TALDO1, PYCR1, R...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
2,KEGG,KEGG:00520,Amino sugar and nucleotide sugar metabolism,0.000575,True,Amino sugar and nucleotide sugar metabolism,44,1348,17,12616,0.012611,0.386364,query_1,[KEGG:00000],"[UGDH, UAP1, GFPT1, UXS1, NANP, GFPT2, GNPNAT1...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
3,KEGG,KEGG:00051,Fructose and mannose metabolism,0.000712,True,Fructose and mannose metabolism,32,1226,13,12616,0.010604,0.406250,query_1,[KEGG:00000],"[PMM2, TKFC, AKR1B10, GMDS, PFKP, PFKL, SORD, ...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
4,KEGG,KEGG:00270,Cysteine and methionine metabolism,0.000922,True,Cysteine and methionine metabolism,47,1068,15,12616,0.014045,0.319149,query_1,[KEGG:00000],"[DNMT3A, AHCYL1, CTH, LDHA, DNMT1, AHCY, PSAT1...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
5,KEGG,KEGG:00250,"Alanine, aspartate and glutamate metabolism",0.003880,True,"Alanine, aspartate and glutamate metabolism",33,890,11,12616,0.012360,0.333333,query_1,[KEGG:00000],"[GPT2, ADSL, PPAT, GFPT1, GFPT2, GLUD1, CAD, G...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
6,KEGG,KEGG:01200,Carbon metabolism,0.008794,True,Carbon metabolism,112,1349,27,12616,0.020015,0.241071,query_1,[KEGG:00000],"[RPIA, GPT2, PGK1, PSPH, TALDO1, RPE, GCSH, GL...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
7,KEGG,KEGG:05230,Central carbon metabolism in cancer,0.023004,True,Central carbon metabolism in cancer,65,953,15,12616,0.015740,0.230769,query_1,[KEGG:00000],"[SLC7A5, SLC1A5, LDHA, MAP2K1, MET, GLS, PGAM1...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
8,KEGG,KEGG:04110,Cell cycle,0.035807,True,Cell cycle,107,130,7,12616,0.053846,0.065421,query_1,[KEGG:00000],"[CDKN2A, MCM6, CCNA2, PLK1, CDK1, CCNB1, CHEK1]","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."


In [14]:
down_reg_results_df = down_reg_results_df[down_reg_results_df.term_size >= 5]
down_reg_results_df = down_reg_results_df[down_reg_results_df.term_size <= 500]
down_reg_results_df = down_reg_results_df[down_reg_results_df.source == 'KEGG']
down_reg_results_df

,source,native,name,p_value,significant,description,term_size,query_size,intersection_size,effective_domain_size,precision,recall,query,parents,intersections,evidences
0,KEGG,KEGG:00010,Glycolysis / Gluconeogenesis,0.002080,True,Glycolysis / Gluconeogenesis,63,111,6,12616,0.054054,0.095238,query_1,[KEGG:00000],"[ALDH3B1, HK3, FBP1, ALDH3A1, PGM1, DLAT]","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG]]"
1,KEGG,KEGG:04979,Cholesterol metabolism,0.011371,True,Cholesterol metabolism,43,26,3,12616,0.115385,0.069767,query_1,[KEGG:00000],"[CD36, CYP27A1, LIPA]","[[KEGG], [KEGG], [KEGG]]"
2,KEGG,KEGG:05412,Arrhythmogenic right ventricular cardiomyopathy,0.011520,True,Arrhythmogenic right ventricular cardiomyopathy,62,44,4,12616,0.090909,0.064516,query_1,[KEGG:00000],"[DMD, ITGA2B, ACTN2, DES]","[[KEGG], [KEGG], [KEGG], [KEGG]]"
3,KEGG,KEGG:05416,Viral myocarditis,0.017889,True,Viral myocarditis,54,27,3,12616,0.111111,0.055556,query_1,[KEGG:00000],"[CAV1, PRF1, DMD]","[[KEGG], [KEGG], [KEGG]]"
4,KEGG,KEGG:05144,Malaria,0.023454,True,Malaria,34,83,4,12616,0.048193,0.117647,query_1,[KEGG:00000],"[CD36, HGF, HBB, PECAM1]","[[KEGG], [KEGG], [KEGG], [KEGG]]"
5,KEGG,KEGG:04145,Phagosome,0.044465,True,Phagosome,137,88,6,12616,0.068182,0.043796,query_1,[KEGG:00000],"[CD36, NCF4, MRC1, HLA-DRB5, NCF2, ATP6V0D1]","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG]]"


In [15]:
up_reg_results_df = up_reg_results_df[up_reg_results_df.term_size >= 5]
up_reg_results_df = up_reg_results_df[up_reg_results_df.term_size <= 500]
up_reg_results_df = up_reg_results_df[up_reg_results_df.source == 'KEGG']
up_reg_results_df

,source,native,name,p_value,significant,description,term_size,query_size,intersection_size,effective_domain_size,precision,recall,query,parents,intersections,evidences
0,KEGG,KEGG:01230,Biosynthesis of amino acids,0.000199,True,Biosynthesis of amino acids,73,1034,20,12616,0.019342,0.273973,query_1,[KEGG:00000],"[RPIA, GPT2, PGK1, CTH, PSPH, TALDO1, PYCR1, R...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
1,KEGG,KEGG:00270,Cysteine and methionine metabolism,0.000451,True,Cysteine and methionine metabolism,47,1007,15,12616,0.014896,0.319149,query_1,[KEGG:00000],"[DNMT3A, AHCYL1, CTH, LDHA, DNMT1, AHCY, PSAT1...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
2,KEGG,KEGG:04115,p53 signaling pathway,0.000771,True,p53 signaling pathway,60,960,18,12616,0.018750,0.300000,query_1,[KEGG:00000],"[CDK1, CCNB1, CHEK1, SERPINE1, GTSE1, SERPINB5...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
3,KEGG,KEGG:00250,"Alanine, aspartate and glutamate metabolism",0.002205,True,"Alanine, aspartate and glutamate metabolism",33,1007,12,12616,0.011917,0.363636,query_1,[KEGG:00000],"[GPT2, ADSL, PPAT, GFPT1, GFPT2, GLUD1, CAD, G...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
4,KEGG,KEGG:05230,Central carbon metabolism in cancer,0.012730,True,Central carbon metabolism in cancer,65,903,15,12616,0.016611,0.230769,query_1,[KEGG:00000],"[SLC7A5, SLC1A5, LDHA, MAP2K1, MET, GLS, PGAM1...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
5,KEGG,KEGG:00520,Amino sugar and nucleotide sugar metabolism,0.018598,True,Amino sugar and nucleotide sugar metabolism,44,937,12,12616,0.012807,0.272727,query_1,[KEGG:00000],"[UGDH, UAP1, GFPT1, UXS1, NANP, GFPT2, GNPNAT1...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
6,KEGG,KEGG:01200,Carbon metabolism,0.019997,True,Carbon metabolism,112,1034,22,12616,0.021277,0.196429,query_1,[KEGG:00000],"[RPIA, GPT2, PGK1, PSPH, TALDO1, RPE, GCSH, GL...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
7,KEGG,KEGG:00051,Fructose and mannose metabolism,0.026186,True,Fructose and mannose metabolism,32,1034,10,12616,0.009671,0.312500,query_1,[KEGG:00000],"[PMM2, TKFC, AKR1B10, GMDS, PFKP, PFKL, SORD, ...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."


Make KEGG Mapper scripts https://www.kegg.jp/kegg/tool/map_pathway2.html Search mode: organism-specific: hsa outside ID: UniProt

In [16]:
for index, row in pval_results_df.iterrows():
    pathway = row.description
    print('\n' + str(pathway))
    old_genes = (row.intersections)
    new_genes = list(gp.convert(query = old_genes, organism = 'hsapiens', target_namespace= 'uniprotswissprot_acc').converted)
    for gene, ID in zip(old_genes, new_genes):
        s = ID
        delta_corr = list(sig_genes[sig_genes.Gene == gene].avg_interaction_coeff)[0]
        if delta_corr > 0:
            s += ' green'
        else:
            s+= ' red'
        print(s)
        
    


Amino sugar and nucleotide sugar metabolism
O60701 green
O94808 green
Q06210 green
Q9BXD5 red
O60547 green
Q8TBE9 green
Q16222 green
O14772 green
Q96EK6 green
Q9UHQ9 green
Q16851 green
P36871 red
O15305 green
Q14376 green
P52790 red
Q8NBZ7 green
P51606 red

Biosynthesis of amino acids
P04075 green
P32322 green
P17174 green
P54687 green
P78330 green
Q8TD30 green
P20132 green
P32929 green
P34897 green
P18669 green
Q96AT9 green
P54886 green
Q9Y617 green
Q01813 green
P00558 green
P00505 green
P04424 green
P14618 green
P17858 green
P37837 green
P49247 green

Fructose and mannose metabolism
P04075 green
O60218 green
P09467 red
O60825 green
O60547 green
O14772 green
Q01813 green
Q00796 green
Q3LXA3 green
O15305 green
P15121 red
P17858 green
P52790 red

Cysteine and methionine metabolism
P17174 green
P54687 green
P23526 green
P20132 green
P32929 green
Q9Y6K1 green
P25325 green
Q96RQ9 green
P26358 green
Q9Y617 green
O43865 green
P48506 green
P00505 green
P48507 green
P00338 green

p53 signalin

In [17]:
for index, row in interaction_results_df.iterrows():
    pathway = row.description
    print('\n' + str(pathway))
    old_genes = (row.intersections)
    new_genes = list(gp.convert(query = old_genes, organism = 'hsapiens', target_namespace= 'uniprotswissprot_acc').converted)
    for gene, ID in zip(old_genes, new_genes):
        s = ID
        delta_corr = list(sig_genes[sig_genes.Gene == gene].avg_interaction_coeff)[0]
        if delta_corr > 0:
            s += ' green'
        else:
            s+= ' red'
        print(s)
        
    


p53 signaling pathway
P42771 red
Q8N726 green
P06493 green
P14635 green
O14757 green
P05121 green
Q9NYZ3 green
P36952 green
P31947 green
P31350 green
O96017 green
P24941 green
P17936 green
Q658P3 green
P55211 green
P07996 green
P60484 green
Q53FA7 green
Q13315 green

Biosynthesis of amino acids
P49247 green
Q8TD30 green
P00558 green
P32929 green
P78330 green
P37837 green
P32322 green
Q96AT9 green
Q9Y617 green
P34897 green
P54886 green
P18669 green
Q01813 green
P04424 green
P17858 green
P14618 green
P17174 green
P20132 green
P00505 green
P04075 green

Amino sugar and nucleotide sugar metabolism
O60701 green
Q16222 green
Q06210 green
Q8NBZ7 green
Q8TBE9 green
O94808 green
Q96EK6 green
O15305 green
O60547 green
Q16851 green
O14772 green
Q14376 green
P52790 red
Q9BXD5 red
Q9UHQ9 green
P51606 red
P36871 red

Fructose and mannose metabolism
O15305 green
Q3LXA3 green
O60218 green
O60547 green
Q01813 green
P17858 green
Q00796 green
O14772 green
O60825 green
P04075 green
P52790 red
P09467 red


In [18]:
for index, row in up_reg_results_df.iterrows():
    pathway = row.description
    print('\n' + str(pathway))
    old_genes = (row.intersections)
    new_genes = list(gp.convert(query = old_genes, organism = 'hsapiens', target_namespace= 'uniprotswissprot_acc').converted)
    for gene, ID in zip(old_genes, new_genes):
        s = ID
        delta_corr = list(sig_genes[sig_genes.Gene == gene].avg_interaction_coeff)[0]
        if delta_corr > 0:
            s += ' green'
        else:
            s+= ' red'
        print(s)
        
    


Biosynthesis of amino acids
P49247 green
Q8TD30 green
P00558 green
P32929 green
P78330 green
P37837 green
P32322 green
Q96AT9 green
Q9Y617 green
P34897 green
P54886 green
P18669 green
Q01813 green
P04424 green
P17858 green
P14618 green
P17174 green
P20132 green
P00505 green
P04075 green

Cysteine and methionine metabolism
Q9Y6K1 green
O43865 green
P32929 green
P00338 green
P26358 green
P23526 green
Q9Y617 green
P48506 green
P48507 green
P17174 green
Q96RQ9 green
Q16762 green
P20132 green
P25325 green
P00505 green

p53 signaling pathway
P06493 green
P14635 green
O14757 green
P05121 green
Q9NYZ3 green
P36952 green
P31947 green
P31350 green
O96017 green
P24941 green
P17936 green
Q658P3 green
P55211 green
P07996 green
P60484 green
Q53FA7 green
Q13315 green
P42574 green

Alanine, aspartate and glutamate metabolism
Q8TD30 green
P30566 green
Q06203 green
Q06210 green
O94808 green
P00367 green
P27708 green
O94925 green
P04424 green
P17174 green
Q96RQ9 green
P00505 green

Central carbon metabo

In [19]:
for index, row in down_reg_results_df.iterrows():
    pathway = row.description
    print('\n' + str(pathway))
    old_genes = (row.intersections)
    new_genes = list(gp.convert(query = old_genes, organism = 'hsapiens', target_namespace= 'uniprotswissprot_acc').converted)
    for gene, ID in zip(old_genes, new_genes):
        s = ID
        delta_corr = list(sig_genes[sig_genes.Gene == gene].avg_interaction_coeff)[0]
        if delta_corr > 0:
            s += ' green'
        else:
            s+= ' red'
        print(s)
        
    


Glycolysis / Gluconeogenesis
P43353 red
P52790 red
P09467 red
P30838 red
P36871 red
P10515 red

Cholesterol metabolism
P16671 red
Q02318 red
P38571 red

Arrhythmogenic right ventricular cardiomyopathy
P11532 red
P08514 red
P35609 red
P17661 red

Viral myocarditis
Q03135 red
P14222 red
P11532 red

Malaria
P16671 red
P14210 red
P68871 red
P16284 red

Phagosome
P16671 red
Q15080 red
P22897 red
Q30154 red
P19878 red
P61421 red


In [20]:
# look at differences between protein expression levels of tRNA synthase in tumor vs normal
# does it matter which way to do pathway analysis? 
# change cut off to 15 good sample points for each